In [13]:
import cvxpy as cp
import numpy as np
import pandas as pd

In [14]:
def load_and_split_data(train_path="train.csv", test_path="test.csv"):
    """
    Load data from CSV files and split into training, validation, and test sets.
    
    Parameters:
    - train_path: Path to the training CSV file.
    - test_path: Path to the testing CSV file.
    
    Returns:
    - train_data, train_labels: Training data and labels.
    - val_data, val_labels: Validation data and labels.
    - test_data, test_labels: Test data and labels.
    """
    
    # Load train.csv and split
    train_df = pd.read_csv(train_path, header=None)

    train_data = train_df.iloc[:4000, 1:].values
    train_labels = train_df.iloc[:4000, 0].values

    # print("The first 10 labels in the training set are: ", train_labels[:10])
    # print("The first 10 features in the training set are: ", train_data[:10])    
    
    val_data = train_df.iloc[4000:, 1:].values
    val_labels = train_df.iloc[4000:, 0].values
    
    # Load test.csv
    test_df = pd.read_csv(test_path, header=None)
    test_data = test_df.iloc[:, 1:].values
    test_labels = test_df.iloc[:, 0].values

    test_labels = test_labels * 2 - 1
    val_labels = val_labels * 2 - 1
    train_labels = train_labels * 2 - 1
    
    return train_data, train_labels, val_data, val_labels, test_data, test_labels

# Load data
train_data, train_labels, val_data, val_labels, test_data, test_labels = load_and_split_data()



# Q2

In [15]:
def svm_train_primal(data_train, label_train, regularisation_para_C):    
    N, d = data_train.shape
    w = cp.Variable(d)
    b = cp.Variable()
    xi = cp.Variable(N)
    
    # Objective function
    objective = cp.Minimize(0.5 * cp.norm(w,2)**2 + (regularisation_para_C/N) * cp.sum(xi))
    
    # Constraints
    constraints = [cp.multiply(label_train, data_train @ w + b) >= 1 - xi, xi >= 0]
    
    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    svm_model = {'w': w.value, 'b': b.value}
    
    return svm_model


In [16]:
def svm_predict_primal(data_test, label_test, svm_model):
    # Extract w and b from the model
    w = svm_model['w']
    b = svm_model['b']

    # Predict
    preds = np.sign(np.dot(data_test, w) + b)
    
    # Calculate accuracy
    test_accuracy = np.mean(preds == label_test)
    return test_accuracy


In [30]:
svm_model = svm_train_primal(train_data, train_labels, 100)
#print b and sum of w
print(f"b: {svm_model['b']}")
print(f"sum of w: {np.sum(svm_model['w'])}")
#test for the accuraccy
test_accuracy = svm_predict_primal(test_data, test_labels, svm_model)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

b: 1.779813717087077
sum of w: -0.1452156803361282
Test Accuracy: 96.80%


# Q3

In [18]:
def svm_train_dual(data_train, label_train, regularisation_para_C):
    # regularization_term = 0
    N,_ = data_train.shape
    alpha = cp.Variable((N,1))

    # Reshape label_train to be a column vector
    label_train = label_train.reshape(-1, 1)
    mult = label_train * data_train

    mult_transpose = mult.T

    # Construct the kernel (gram matrix) with regularization\
    K = cp.sum_squares(mult_transpose @ alpha)
    
    # Objective
    objective = cp.Minimize(-1 * cp.sum(alpha) + K)
    
    # Constraints
    constraints = [
        cp.sum(cp.multiply(alpha, label_train)) == 0,
        alpha >= 0,
        alpha <= regularisation_para_C / N
    ]
    
    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    return alpha.value


In [19]:
alpha = svm_train_dual(train_data, train_labels, 100)
print("Dual: sum of alpha: ", np.sum(alpha))

Dual: sum of alpha:  6.477017459325738


C:\Users\77837\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


# Q4

In [20]:
def compute_primal_from_dual(alpha, data_train, label_train, regularisation_para_C):
    """
    Compute the primal SVM solution (w*, b*) from the dual solution alpha*.

    Parameters:
    - alpha: Dual solution (N,)
    - data_train: Training data matrix (N x D)
    - label_train: Training labels (N,)
    - regularisation_para_C: Regularization parameter C

    Returns:
    - w: Weight vector (D,)
    - b: Bias scalar
    """
    # Compute w* from alpha*
    print((alpha * label_train).shape)
    w = np.dot((alpha * label_train).T, data_train)

    # Compute b* using a support vector
    # Find a support vector index (any example where 0 < alpha < C/N)
    support_vector_indices = np.where((alpha > 1e-5) & (alpha < (regularisation_para_C / data_train.shape[0])))[0]
    if len(support_vector_indices) > 0:
        sv_index = support_vector_indices[0]
        b = label_train[sv_index] - np.dot(w, data_train[sv_index])
    else:
        b = 0

    return w, b

w, b = compute_primal_from_dual(alpha, train_data, train_labels, 100)
print("Primal from Dual:\tsum of w: ", np.sum(w))
print("Primal from Dual:\tb: ", b)

(4000, 4000)
Primal from Dual:	sum of w:  3092.3424254572155
Primal from Dual:	b:  [-55.85478171  53.85478171  53.85478171 ...  53.85478171  53.85478171
  53.85478171]


# Q5

In [21]:
def find_support_vectors_from_primal(data_train, label_train, svm_model):
    """
    Find the support vectors based on the primal SVM model.

    Parameters:
    - data_train: Training data matrix (N x D)
    - label_train: Training labels (N,)
    - svm_model: Trained SVM model with 'w' and 'b' as keys

    Returns:
    - support_vectors: Matrix of support vectors (S x D)
    - support_vector_labels: Labels of the support vectors (S,)
    - support_vector_indices: Indices of the support vectors in the training data
    """
    # Extract w and b from the model
    w = svm_model['w']
    b = svm_model['b']

    # Compute decision values
    decision_values = label_train * (np.dot(data_train, w) + b)
    
    # Identify support vectors
    support_vector_indices = np.where(decision_values <= 1)[0]
    support_vectors = data_train[support_vector_indices]
    support_vector_labels = label_train[support_vector_indices]

    return support_vectors, support_vector_labels, support_vector_indices

# Test the function using the provided SVM model (assuming it's available in the notebook)
# This step assumes that the variables train_data, train_labels, and svm_model are already loaded.
# svm_model = {"w": w, "b": b}
# support_vectors, support_vector_labels, support_vector_indices = find_support_vectors_from_primal(train_data, train_labels, svm_model)
# len(support_vector_indices)  # This will display the number of support vectors found


# Q6

In [22]:
def find_support_vectors_from_dual(data_train, label_train, alpha_values, threshold=1e-5):
    """
    Find the support vectors based on the dual SVM solution.

    Parameters:
    - data_train: Training data matrix (N x D)
    - label_train: Training labels (N,)
    - alpha_values: Values of the Lagrange multipliers from the dual solution (N,)
    - threshold: Threshold for considering a data point as a support vector

    Returns:
    - support_vectors: Matrix of support vectors (S x D)
    - support_vector_labels: Labels of the support vectors (S,)
    - support_vector_indices: Indices of the support vectors in the training data
    - support_vector_alphas: Values of the Lagrange multipliers for the support vectors
    """
    # Identify support vectors based on alpha values
    support_vector_indices = np.where(alpha_values > threshold)[0]
    support_vectors = data_train[support_vector_indices]
    support_vector_labels = label_train[support_vector_indices]
    support_vector_alphas = alpha_values[support_vector_indices]

    return support_vectors, support_vector_labels, support_vector_indices, support_vector_alphas


## Q7 (BEST C IS 4)

In [31]:
def select_best_C(train_data, train_labels, val_data, val_labels):
    """
    Choose the best value of C using the validation set.

    Parameters:
    - train_data: Training data matrix
    - train_labels: Training labels
    - val_data: Validation data matrix
    - val_labels: Validation labels

    Returns:
    - best_C: Optimal value of C
    - best_accuracy: Accuracy on the validation set using the best C
    """
    C_values = [2**i for i in range(-2, 11)]
    best_accuracy = -0.1
    best_C = None
    
    for C in C_values:
        # print("Training for C={}".format(C))
        # Train SVM using the current value of C
        svm_model = svm_train_primal(train_data, train_labels, C)
        # print(svm_model)

        accuracy = svm_predict_primal(val_data, val_labels, svm_model)

        # print("Accuracy for C={}: {}".format(C, accuracy))
        
        if accuracy > best_accuracy:
            # print("Found better value of C: {} with accuracy: {}".format(C, accuracy))
            best_accuracy = accuracy
            best_C = C
            
    return best_C, best_accuracy

# Using the above function
best_C, best_val_accuracy = select_best_C(train_data, train_labels, val_data, val_labels)
print(f"Best C: {best_C}, Validation Accuracy: {best_val_accuracy * 100:.2f}%")

# Train the SVM using the best C and report test accuracy
svm_model = svm_train_primal(train_data, train_labels, best_C)

test_accuracy = svm_predict_primal(test_data, test_labels, svm_model)
print(f"Test Accuracy using best C: {test_accuracy * 100:.2f}%")

Best C: 4, Validation Accuracy: 97.49%
Test Accuracy using best C: 97.47%


# Q8

In [32]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

def sklearn_svm():

    #CREAT A SVC model, with C= 4
    svc_model = Pipeline([('svc', SVC(C=4, kernel='linear'))])

    #fit the model using the pipeline
    svc_model.fit(train_data, train_labels)

    #predict the model
    svc_preds = svc_model.predict(test_data)

    #compute the accuracy
    svc_accuracy = np.mean(svc_preds == test_labels)
    print(f"Test Accuracy using best C: {svc_accuracy * 100:.2f}%")
    return svc_accuracy

# Q8

In [33]:
sklearn_svm()

Test Accuracy using best C: 96.13%


0.9613333333333334

# Q7

In [34]:
# Using the above function
best_C, best_val_accuracy = select_best_C(train_data, train_labels, val_data, val_labels)
print(f"Best C: {best_C}, Validation Accuracy: {best_val_accuracy * 100:.2f}%")

# Train the SVM using the best C and report test accuracy
svm_model = svm_train_primal(train_data, train_labels, best_C)

test_accuracy = svm_predict_primal(test_data, test_labels, svm_model)
print(f"Test Accuracy using best C: {test_accuracy * 100:.2f}%")

Best C: 4, Validation Accuracy: 97.49%
Test Accuracy using best C: 97.47%


# Q3

# Q4

In [35]:
w, b = compute_primal_from_dual(alpha, train_data, train_labels, 100)
print("Primal from Dual:\tsum of w: ", np.sum(w))
print("Primal from Dual:\tb: ", b)

(4000, 4000)
Primal from Dual:	sum of w:  3092.3424254572155
Primal from Dual:	b:  [-55.85478171  53.85478171  53.85478171 ...  53.85478171  53.85478171
  53.85478171]


# Q5

In [28]:
support_vectors, support_vector_labels, support_vector_indices = find_support_vectors_from_primal(train_data, train_labels, svm_model)
print("Number of support vectors: ", len(support_vector_indices))
print("Support vector indices: ", support_vector_indices)

Number of support vectors:  2436
Support vector indices:  [   0    3    4 ... 3996 3997 3998]


# Q6

In [29]:
support_vectors, support_vector_labels, support_vector_indices, support_vector_alphas = find_support_vectors_from_dual(train_data, train_labels, alpha)
print("Number of support vectors: ", len(support_vector_indices))
print("Support vector indices: ", support_vector_indices)

Number of support vectors:  372
Support vector indices:  [   0   28   29   41   54   58   68   80   88  127  140  145  147  150
  163  208  241  253  263  266  275  296  305  307  332  344  356  377
  395  400  401  403  405  414  420  424  432  433  441  446  450  473
  479  495  521  525  547  561  563  564  567  576  581  587  595  604
  610  637  642  656  660  678  703  708  709  736  737  752  768  774
  796  799  809  816  821  826  834  843  856  861  873  905  909  930
  932  935  937  948  965  967  983  988 1001 1003 1015 1017 1022 1050
 1067 1082 1098 1116 1123 1139 1142 1166 1167 1178 1179 1188 1219 1242
 1266 1274 1295 1304 1320 1364 1371 1374 1377 1403 1406 1407 1414 1416
 1440 1447 1450 1474 1475 1476 1478 1489 1492 1493 1531 1536 1539 1555
 1582 1590 1598 1616 1627 1635 1636 1651 1659 1664 1667 1675 1680 1683
 1684 1691 1692 1705 1706 1724 1737 1775 1778 1780 1799 1825 1829 1833
 1840 1848 1851 1852 1854 1862 1864 1865 1869 1884 1886 1889 1890 1900
 1912 1921 1924 1957